# Senitment Prediction for Robo Eye Animation

The project, involves a dynamic robot eye animation displayed on a 0.96" OLED screen that changes based on the sentiment of text input provided by the user. The hardware controller used for this project is the CAP10 Pratham, a made-in-India board. This project offers a unique way to integrate emotional intelligence into robotics, enabling bots to visually express emotions in response to user interactions.

**Author:** Asutosh Pati ([https://www.linkedin.com/in/asutoshpati/](https://www.linkedin.com/in/asutoshpati/))  
**Date:** 21-Aug-2024  
**Versions:**  
- V1.0: Initial Release
- V1.1: Change communication method from Serial to API call- V2.0: Add text to speech functionality

### Install & Import Modules

In [1]:
# !pip install tensorflow==2.13.1

In [2]:
import os
import pickle
import re
import time

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [3]:
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package omw-1.4 to /home/santosh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/santosh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Preprocess the sentence

In [4]:
stop_words = set(stopwords.words("english"))
lemmatizer= WordNetLemmatizer()

In [5]:
def lower_case(text):
    text = text.split()
    text = [y.lower() for y in text]
    return " ".join(text)

lower_case("Oh, Yes! I have ssen that.")

'oh, yes! i have ssen that.'

In [6]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

remove_urls("Have you visited https://www.google.co.in/")

'Have you visited '

In [7]:
def remove_punctuations(text):
    # Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )
    
    # remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

remove_punctuations("Are you sure; It's amazing!")

'Are you sure It s amazing'

In [8]:
def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

remove_numbers("Have you seen the prime minister's speech at 10 am")

"Have you seen the prime minister's speech at  am"

In [9]:
def remove_stop_words(text):
    text = [i for i in str(text).split() if i not in stop_words]
    return " ".join(text)

remove_stop_words("I am a very good boy; is it true")

'I good boy; true'

In [10]:
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(y) for y in text]
    return " ".join(text)

lemmatization("I love to watch Footbal. What do you like to do?")

'I love to watch Footbal. What do you like to do?'

In [11]:
def preprocess_sentence(sentence):
    sentence = lower_case(sentence)
    sentence = remove_urls(sentence)
    sentence = remove_punctuations(sentence)
    sentence = remove_numbers(sentence)
    sentence = remove_stop_words(sentence)
    sentence = lemmatization(sentence)
    return sentence

preprocess_sentence("Arise, awake, and stop not until the goal is reached. - Swami Vivekananda")

'arise awake stop goal reached swami vivekananda'

### Load Models

In [12]:
# import label encoder created during training

le = None
with open("./model/label_encoder.pkl",'rb') as file:
    le = pickle.load(file)

/home/santosh/.conda/envs/llm/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
# import tokenizer created during training

tokenizer = None
with open("./model/tokenizer.json") as file:
    tokenizer_json = file.read()
    tokenizer = tokenizer_from_json(tokenizer_json)

In [14]:
# import the pre-trained model

model_path = "./model/Sentiment_analysis_Eng-V3.h5"
model = load_model(model_path)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 229, 200)          2865000   
                                                                 
 bidirectional (Bidirection  (None, 229, 512)          935936    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 229, 256)          656384    
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 229, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 229, 128)          164352    
 onal)                                                  

### Use the model

In [15]:
# Labels ['fear', 'sadness', 'joy', 'anger', 'love', 'surprise']

In [16]:
def extract_emotion(text):
    sentence = preprocess_sentence(text)
    
    sentence = tokenizer.texts_to_sequences([sentence])
    sentence = pad_sequences(sentence, maxlen=229, truncating='pre')
    
    result = le.inverse_transform(np.argmax(model.predict(sentence), axis=-1))
    proba =  np.max(model.predict(sentence))

    # class_predict =  np.max(model.predict(sentence))
    # print(class_predict)
    
    print(f"{result} : {proba}\n\n")
    
    return result[0]

extract_emotion("Hurray!! My Model Got 93% Accuracy, Connect With me https://www.linkedin.com/in/asutoshpati/")

1/1 [==============================] - 1s 585ms/step
['joy'] : 0.3245931565761566




'joy'

In [17]:
example_sentences = [
    "He's over the moon about being accepted to the university",
    "Your point on this certain matter made me outrageous, how can you say so? This is insane.",
    "I can't do it, I'm not ready to lose anything, just leave me alone",
    "Merlin's beard harry, you can cast the Patronus charm! I'm amazed!",
    "I am amazed, that you do it."
]

In [18]:
for sentence in example_sentences:
    print(sentence)
    print(extract_emotion(sentence))
    print("\n\n")

He's over the moon about being accepted to the university
1/1 [==============================] - 1s 591ms/step
['joy'] : 0.8274199962615967


joy



Your point on this certain matter made me outrageous, how can you say so? This is insane.
1/1 [==============================] - 1s 576ms/step
['anger'] : 0.4897823929786682


anger



I can't do it, I'm not ready to lose anything, just leave me alone
1/1 [==============================] - 1s 597ms/step
['fear'] : 0.3876216411590576


fear



Merlin's beard harry, you can cast the Patronus charm! I'm amazed!
1/1 [==============================] - 1s 594ms/step
['surprise'] : 0.935206949710846


surprise



I am amazed, that you do it.
1/1 [==============================] - 1s 594ms/step
['surprise'] : 0.29983246326446533


surprise





In [19]:
# !pip install pyaudio
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# !pip install --upgrade transformers datasets[audio] accelerate
# !pip install whisperx


In [20]:
import audioop
import gc
import os
import pyaudio
import torch
import uuid
import wave
import whisperx

/tmp/ipykernel_34940/3608104360.py:1: DeprecationWarning: 'audioop' is deprecated and slated for removal in Python 3.13
  import audioop
/home/santosh/.conda/envs/llm/lib/python3.11/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [21]:
# Whisper Model Parameters
WHISPER_MODEL_NAME = "small.en"
COMPUTE_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# use "float16" for GPU & "int8" for CPU
COMPUTE_DTYPE = "float16" if COMPUTE_DEVICE == "cuda" else "int8"
SUPRESS_NUMERALS = True
BATCH_SIZE = 16

In [22]:
RECORD_DIR ="./recordings"

# Create a directory to store recordings if it doesn't exist
if not os.path.exists(RECORD_DIR):
    os.makedirs(RECORD_DIR)

In [23]:
# Load the whisper model
whisper_model = whisperx.load_model(
    WHISPER_MODEL_NAME,
    COMPUTE_DEVICE,
    compute_type=COMPUTE_DTYPE,
    # asr_options={"suppress_numerals": SUPRESS_NUMERALS}
)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.1+cpu. Bad things might happen unless you revert torch to 1.x.


In [24]:
def normalize_audio(frames, target_rms=1000):
    # Adjust audio frames to a target RMS for improved sound quality
    rms = audioop.rms(b''.join(frames), 2)
    gain = target_rms / (rms + 1)  # Avoid division by zero
    normalized_frames = [audioop.mul(frame, 2, gain) for frame in frames]
    return normalized_frames

In [25]:
def record_audio(silence_threshold=1000, silence_duration=2):
    chunk = 1024
    audio_format = pyaudio.paInt16
    channels = 1
    rate = 44100

    p = pyaudio.PyAudio()
    stream = p.open(format=audio_format, channels=channels, rate=rate, input=True, frames_per_buffer=chunk)

    print("Recording...")
    frames = []
    silence_count = 0

    while True:
        data = stream.read(chunk)
        frames.append(data)
        rms = audioop.rms(data, 2)
        if rms < silence_threshold:
            silence_count += 1
        else:
            silence_count = 0

        if silence_count >= silence_duration * (rate / chunk):
            print("Silence detected. Stopping recording.")
            break

    stream.stop_stream()
    stream.close()
    p.terminate()

    # Normalize audio
    normalized_frames = normalize_audio(frames)

    output_filename = str(uuid.uuid4()) + ".wav"
    output_filename = os.path.join(os.getcwd(), RECORD_DIR, output_filename)

    with wave.open(output_filename, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(audio_format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(normalized_frames))

    print(f"Audio saved to {output_filename}")
    return output_filename

In [26]:
def process_audio(audio_file):
    global BATCH_SIZE
    global whisper_model

    audio = whisperx.load_audio(audio_file)
    result = whisper_model.transcribe(audio, batch_size=BATCH_SIZE)
    # print(result["segments"])  # before alignment

    gc.collect()
    torch.cuda.empty_cache()
    # del whisper_model

    return result["segments"]

In [27]:
def get_text_from_transcribe(processed_transcribe):
    return " ".join([sentence["text"] for sentence in processed_transcribe])

In [28]:
# Test out the speech to text code# recorded_file = record_audio(silence_threshold=500, silence_duration=2)
# transcribe = process_audio(recorded_file)
# final_text = get_text_from_transcribe(transcribe)
# print(final_text)

In [29]:
def speech_to_text_process():
    recorded_file = record_audio(silence_threshold=500, silence_duration=2)
    transcribe = process_audio(recorded_file)
    final_text = get_text_from_transcribe(transcribe)
    return final_text

In [30]:
emotion_mood_mapping = {"fear": "F", "sadness": "S", "joy": "H", "anger": "A", "love": "L", "surprise": "N"}

In [31]:
import requests

def send_mood(mood):
    url = 'http://192.168.1.1'

    try:
        payload = {'mood': mood}
        response = requests.get(url, params=payload, timeout=2)

        if response.status_code == 200:
            print("Request sent to bot")
            return None
        else:
            print('Error:', response.status_code)
            return None
    except requests.exceptions.RequestException as e:
        print('Error:', e)
        return None

In [32]:
# Few examples that you can try
# Hey, I have a good news for you. are you amazed! - surprise
# Its your birthday, I have gift for you. aren't you happy. - joy
# What you've done has left me heartbroken. - sadness
# Are you afraid that I know what you have done ? - fear
# For your nasty work I am going to scold you - anger

In [33]:
# Don't forget to connect with CAP10 before running this cell
# Test with user input text

# text = input("Enter your text: ")
# emotion = extract_emotion(text)
# mood = emotion_mood_mapping.get(emotion)
# send_mood(mood)

In [34]:
LISTENING_DELAY = 10
while True:
    text = speech_to_text_process()
    print(text)
    emotion = extract_emotion(text)
    print(emotion)
    mood = emotion_mood_mapping.get(emotion)
    print(mood)
    # send_mood(mood)  # Send information to bot
    time.sleep(LISTENING_DELAY)

ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


Recording...
Silence detected. Stopping recording.
Audio saved to /home/santosh/sentiment/Ani-Emo-Eye-master/./recordings/d884d34e-0547-4bf1-bf0d-610a0aad19dc.wav
 The cake taste delicious
1/1 [==============================] - 1s 584ms/step
['joy'] : 0.9997907280921936


joy
H


ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


Recording...
Silence detected. Stopping recording.
Audio saved to /home/santosh/sentiment/Ani-Emo-Eye-master/./recordings/311a1d0e-9cde-4ab5-9cf7-978c20244bd0.wav
 You have got to have a positive attitude to do well in life.
1/1 [==============================] - 1s 607ms/step
['joy'] : 0.5851826667785645


joy
H


ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


...

ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


Recording...
Silence detected. Stopping recording.
Audio saved to /home/santosh/sentiment/Ani-Emo-Eye-master/./recordings/b000d68e-4390-4e31-bb55-9d749cba91c7.wav
 What a nice surprise.
1/1 [==============================] - 1s 610ms/step
['surprise'] : 0.7299973964691162


surprise
N


ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


Recording...


KeyboardInterrupt: 